# just the s box

In [3]:
import gurobipy as gp
from gurobipy import GRB

# 4-bit S-box definition
s0 = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]

# Compute valid (Δx, Δy) pairs
valid_pairs = set()
for dx in range(16):
    for x in range(16):
        dy = s0[x] ^ s0[x ^ dx]
        valid_pairs.add((dx, dy))

# All possible pairs
all_pairs = {(dx, dy) for dx in range(16) for dy in range(16)}
invalid_pairs = all_pairs - valid_pairs

def bits4(n):
    """Return 4-bit binary representation (LSB first)."""
    return [(n >> i) & 1 for i in range(4)]

# MILP model
m = gp.Model("Sbox_BranchNumber")

# Binary variables for input/output difference bits
x = m.addVars(4, vtype=GRB.BINARY, name="x")
y = m.addVars(4, vtype=GRB.BINARY, name="y")

# Objective: minimize Hamming weight of input and output differences
m.setObjective(sum(x[i] for i in range(4)) + sum(y[j] for j in range(4)), GRB.MINIMIZE)

# Constraint: input difference ≠ 0
m.addConstr(sum(x[i] for i in range(4)) >= 1, "nonzero_dx")

# Forbid invalid differential transitions
for dx, dy in invalid_pairs:
    dxb = bits4(dx)
    dyb = bits4(dy)
    expr = gp.LinExpr()
    for i in range(4):
        expr += x[i] if dxb[i] == 1 else (1 - x[i])
    for j in range(4):
        expr += y[j] if dyb[j] == 1 else (1 - y[j])
    m.addConstr(expr <= 7)

# Solve MILP
m.optimize()

# Display result
if m.status == GRB.OPTIMAL:
    dx_bits = [int(x[i].x) for i in range(4)]
    dy_bits = [int(y[j].x) for j in range(4)]
    dx_val = sum(b << i for i, b in enumerate(dx_bits))
    dy_val = sum(b << i for i, b in enumerate(dy_bits))
    print("\n=== MILP Result ===")
    print(f"Optimal branch number: {m.objVal}")
    print(f"Δx = {dx_val:04b} ({dx_val}), Δy = {dy_val:04b} ({dy_val})")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i5-12450HX, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 151 rows, 8 columns and 1204 nonzeros
Model fingerprint: 0x81d2ef70
Variable types: 0 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 105 rows and 0 columns
Presolve time: 0.01s
Presolved: 46 rows, 8 columns, 270 nonzeros
Variable types: 0 continuous, 8 integer (8 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

# one round all constriants

In [6]:
import gurobipy as gp
from gurobipy import GRB

# ======================= S-box definition =======================
s0 = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]

def bits4(n):
    return [(n >> i) & 1 for i in range(4)]

valid_pairs = set()
for dx in range(16):
    for x in range(16):
        dy = s0[x] ^ s0[x ^ dx]
        valid_pairs.add((dx, dy))
all_pairs = {(dx, dy) for dx in range(16) for dy in range(16)}
invalid_pairs = all_pairs - valid_pairs


# ======================= Utility: XOR constraints =======================
def add_xor2(m, z, a, b, name_prefix="xor"):
    m.addConstr(z <= a + b, name=f"{name_prefix}_1")
    m.addConstr(z >= a - b, name=f"{name_prefix}_2")
    m.addConstr(z >= b - a, name=f"{name_prefix}_3")
    m.addConstr(z <= 2 - (a + b), name=f"{name_prefix}_4")


# ======================= MDS definition =======================
def mul2_pair(m, t0, t1, name):
    """Implements t0,t1 = mul2(t0,t1)"""
    new0 = {b: m.addVar(vtype=GRB.BINARY, name=f"{name}_new0_{b}") for b in range(4)}
    new1 = {b: m.addVar(vtype=GRB.BINARY, name=f"{name}_new1_{b}") for b in range(4)}
    for b in range(4):
        m.addConstr(new0[b] == t1[b])
        add_xor2(m, new1[b], a=t0[b], b=t1[b], name_prefix=f"{name}_xor_{b}")
    return new0, new1


def add_MDS_constraints(m, xin, xout):
    """Add Saturnin MDS linear layer between xin[0..7][4bits] and xout."""
    # Split into nibbles A,B,C,D
    A0 = {b: xin[(0,b)] for b in range(4)}
    A1 = {b: xin[(1,b)] for b in range(4)}
    B0 = {b: xin[(2,b)] for b in range(4)}
    B1 = {b: xin[(3,b)] for b in range(4)}
    C0 = {b: xin[(4,b)] for b in range(4)}
    C1 = {b: xin[(5,b)] for b in range(4)}
    D0 = {b: xin[(6,b)] for b in range(4)}
    D1 = {b: xin[(7,b)] for b in range(4)}

    # Step 1: C ^= D; A ^= B
    C0_1, C1_1, A0_1, A1_1 = {}, {}, {}, {}
    for b in range(4):
        C0_1[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, C0_1[b], C0[b], D0[b], "C0_1")
        C1_1[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, C1_1[b], C1[b], D1[b], "C1_1")
        A0_1[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, A0_1[b], A0[b], B0[b], "A0_1")
        A1_1[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, A1_1[b], A1[b], B1[b], "A1_1")

    # Step 2: MUL2 on B,D
    B0_2, B1_2 = mul2_pair(m, B0, B1, "B")
    D0_2, D1_2 = mul2_pair(m, D0, D1, "D")

    # Step 3: B ^= C; D ^= A
    B0_3, B1_3, D0_3, D1_3 = {}, {}, {}, {}
    for b in range(4):
        B0_3[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, B0_3[b], B0_2[b], C0_1[b], "B0_3")
        B1_3[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, B1_3[b], B1_2[b], C1_1[b], "B1_3")
        D0_3[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, D0_3[b], D0_2[b], A0_1[b], "D0_3")
        D1_3[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, D1_3[b], D1_2[b], A1_1[b], "D1_3")

    # Step 4: A = MUL2²(A); C = MUL2²(C)
    A0_t, A1_t = mul2_pair(m, A0_1, A1_1, "A_step1")
    A0_4, A1_4 = mul2_pair(m, A0_t, A1_t, "A_step2")
    C0_t, C1_t = mul2_pair(m, C0_1, C1_1, "C_step1")
    C0_4, C1_4 = mul2_pair(m, C0_t, C1_t, "C_step2")

    # Step 5: final XORs (C^=D, A^=B, B^=C, D^=A)
    C0_f, C1_f, A0_f, A1_f, B0_f, B1_f, D0_f, D1_f = {}, {}, {}, {}, {}, {}, {}, {}
    for b in range(4):
        C0_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, C0_f[b], C0_4[b], D0_3[b], "C0_f")
        C1_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, C1_f[b], C1_4[b], D1_3[b], "C1_f")
        A0_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, A0_f[b], A0_4[b], B0_3[b], "A0_f")
        A1_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, A1_f[b], A1_4[b], B1_3[b], "A1_f")
        B0_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, B0_f[b], B0_3[b], C0_f[b], "B0_f")
        B1_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, B1_f[b], B1_3[b], C1_f[b], "B1_f")
        D0_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, D0_f[b], D0_3[b], A0_f[b], "D0_f")
        D1_f[b] = m.addVar(vtype=GRB.BINARY); add_xor2(m, D1_f[b], D1_3[b], A1_f[b], "D1_f")

    # connect to output
    for b in range(4):
        m.addConstr(xout[(0,b)] == A0_f[b])
        m.addConstr(xout[(1,b)] == A1_f[b])
        m.addConstr(xout[(2,b)] == B0_f[b])
        m.addConstr(xout[(3,b)] == B1_f[b])
        m.addConstr(xout[(4,b)] == C0_f[b])
        m.addConstr(xout[(5,b)] == C1_f[b])
        m.addConstr(xout[(6,b)] == D0_f[b])
        m.addConstr(xout[(7,b)] == D1_f[b])


# ======================= SR_slice and inverse =======================
def add_SR_slice(m, xin, xout):
    """Apply abcd→badc to nibbles 4..7."""
    for i in range(4, 8):
        # bit permutation: [a,b,c,d] → [b,a,d,c]
        m.addConstr(xout[(i,3)] == xin[(i,2)])
        m.addConstr(xout[(i,2)] == xin[(i,3)])
        m.addConstr(xout[(i,1)] == xin[(i,0)])
        m.addConstr(xout[(i,0)] == xin[(i,1)])
    # copy untouched nibbles
    for i in range(0, 4):
        for b in range(4):
            m.addConstr(xout[(i,b)] == xin[(i,b)])


def add_inv_SR_slice(m, xin, xout):
    """Inverse mapping: badc→abcd"""
    for i in range(4, 8):
        # inverse: [b,a,d,c] → [a,b,c,d]
        m.addConstr(xout[(i,3)] == xin[(i,2)])
        m.addConstr(xout[(i,2)] == xin[(i,3)])
        m.addConstr(xout[(i,1)] == xin[(i,0)])
        m.addConstr(xout[(i,0)] == xin[(i,1)])
    for i in range(0, 4):
        for b in range(4):
            m.addConstr(xout[(i,b)] == xin[(i,b)])


# ======================= Build full MILP model =======================
def build_model():
    m = gp.Model("Saturnin_FullTrail")

    # Define all intermediate states
    states = [{(i,b): m.addVar(vtype=GRB.BINARY, name=f"x{r}_{i}_{b}")
               for i in range(8) for b in range(4)} for r in range(7)]

    x0,x1,x2,x3,x4,x5,x6 = states

    # --- Input nonzero constraint
    m.addConstr(gp.quicksum(x0[(i,b)] for i in range(8) for b in range(4)) >= 1)

    # --- Layer 1: S-box on x0 → x1 ---
    for s in range(8):
        xin = [x0[(s,b)] for b in range(4)]
        yout = [x1[(s,b)] for b in range(4)]
        for dx, dy in invalid_pairs:
            dxb = bits4(dx)
            dyb = bits4(dy)
            expr = gp.LinExpr()
            for i in range(4):
                expr += xin[i] if dxb[i] else (1 - xin[i])
            for i in range(4):
                expr += yout[i] if dyb[i] else (1 - yout[i])
            m.addConstr(expr <= 7)

    # --- Layer 2: MDS x1 → x2 ---
    add_MDS_constraints(m, x1, x2)

    # --- Layer 3: S-box again x2 → x3 ---
    for s in range(8):
        xin = [x2[(s,b)] for b in range(4)]
        yout = [x3[(s,b)] for b in range(4)]
        for dx, dy in invalid_pairs:
            dxb = bits4(dx)
            dyb = bits4(dy)
            expr = gp.LinExpr()
            for i in range(4):
                expr += xin[i] if dxb[i] else (1 - xin[i])
            for i in range(4):
                expr += yout[i] if dyb[i] else (1 - yout[i])
            m.addConstr(expr <= 7)

    # --- Layer 4: SR_slice x3 → x4 ---
    add_SR_slice(m, x3, x4)

    # --- Layer 5: MDS x4 → x5 ---
    add_MDS_constraints(m, x4, x5)

    # --- Layer 6: inv_SR_slice x5 → x6 ---
    add_inv_SR_slice(m, x5, x6)

    # Objective: minimize weight through all rounds
    obj = gp.quicksum(x0[(i,b)] for i in range(8) for b in range(4)) + \
          gp.quicksum(x6[(i,b)] for i in range(8) for b in range(4))
    m.setObjective(obj, GRB.MINIMIZE)

    return m, states


# ======================= Run & print trail =======================
def print_state(title, vars_dict):
    print(f"\n{title}")
    for i in range(8):
        bits = [int(vars_dict[(i,b)].x) for b in range(4)]
        val = sum(bits[b]<<b for b in range(4))
        print(f"x[{i}] = {bits} (0x{val:X})")


if __name__ == "__main__":
    m, states = build_model()
    m.optimize()

    if m.status == GRB.OPTIMAL:
        print(f"\n=== Full Differential Trail (S-box → MDS → S-box → SR_slice → MDS → inv_SR_slice) ===")
        print(f"Objective (wt(input)+wt(output)) = {m.objVal}")
        for i, st in enumerate(states):
            print_state(f"State after layer {i}:", st)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i5-12450HX, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3281 rows, 448 columns and 21696 nonzeros
Model fingerprint: 0xdf71fd79
Variable types: 0 continuous, 448 integer (448 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 1358 rows and 208 columns
Presolve time: 0.05s
Presolved: 1923 rows, 240 columns, 10294 nonzeros
Variable types: 0 continuous, 240 integer (240 binary)
Found heuristic solution: objective 24.0000000

Root relaxation: objective 1.000000e+00, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

# convex hull finding

In [4]:
from sage.all import *

# Define the S-box
s0 = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]

def bits4(n):
    return [(n >> i) & 1 for i in range(4)]

# Step 1: Collect all valid (dx, dy) points in {0,1}^8
valid_points = []
for dx in range(16):
    for x in range(16):
        dy = s0[x] ^ s0[x ^ dx]
        valid_points.append(bits4(dx) + bits4(dy))

# Remove duplicates
valid_points = [list(p) for p in set(tuple(p) for p in valid_points)]
print(f"Total valid differential pairs: {len(valid_points)}")

# Step 2: Build convex hull polytope
P = Polyhedron(vertices=valid_points)
print(f"Polyhedron dimension: {P.dim()}")
print(f"Number of facets (inequalities): {len(P.inequalities_list())}")

# Step 3: Extract inequalities (list form)
inequalities = []
for ineq in P.inequalities_list():
    # Format: [c, a1, a2, ..., a8]  meaning  c + a·x >= 0  →  a·x <= -c
    const = -ineq[0]                # right-hand side b = -c
    coeffs = ineq[1:]               # coefficients [a1..a8]
    # Convert to integer form (clear denominators)
    denoms = [QQ(c).denominator() for c in coeffs + [const]]
    mult = lcm(denoms)
    coeffs_int = [int(mult * c) for c in coeffs]
    const_int = int(mult * const)
    inequalities.append((coeffs_int, const_int))

print(f"\nExtracted {len(inequalities)} convex hull inequalities:\n")
for a, b in inequalities:
    print(a, "<= ", b)

# (Optional) Save to file for Python MILP integration
with open("sbox_convexhull_inequalities.txt", "w") as f:
    f.write("SBOX_INEQUALITIES = [\n")
    for a, b in inequalities:
        f.write(f"    ({a}, {b}),\n")
    f.write("]\n")
print("\nSaved inequalities to sbox_convexhull_inequalities.txt")


Total valid differential pairs: 106
Polyhedron dimension: 8
Number of facets (inequalities): 222

Extracted 222 convex hull inequalities:

[0, -1, 0, 0, 0, 0, 0, 0] <=  -1
[-1, 0, 0, 0, 0, 0, 0, 0] <=  -1
[0, 0, -1, 0, 0, 0, 0, 0] <=  -1
[0, 0, 0, -1, 0, 0, 0, 0] <=  -1
[0, 0, 0, 0, -1, 0, 0, 0] <=  -1
[0, 0, 0, 0, 0, -1, 0, 0] <=  -1
[0, 0, 0, 0, 0, 0, 0, -1] <=  -1
[-1, 1, -1, 0, -1, -1, 0, -1] <=  -4
[-1, -1, -2, -2, 1, -2, -1, 1] <=  -7
[-1, -1, 0, -1, 1, 0, -1, -1] <=  -4
[-1, 0, -1, -1, 1, -1, -1, 0] <=  -4
[0, 1, -1, -1, 0, -1, -1, -1] <=  -4
[1, 1, -1, -2, -1, -2, -2, -1] <=  -7
[-1, 1, -1, 0, 0, -1, -1, -1] <=  -4
[0, -1, -1, -1, 0, -1, -1, 1] <=  -4
[1, 0, -1, -1, -1, -1, -1, 0] <=  -4
[-1, 1, -2, -1, 1, -1, -2, -1] <=  -6
[-2, 1, -2, 1, -1, -1, -1, -2] <=  -7
[0, 0, 0, 0, 0, 0, -1, 0] <=  -1
[-1, 0, -1, -1, 1, 0, -1, -1] <=  -4
[-2, -1, -1, -1, 1, 1, -2, -2] <=  -7
[-1, 1, -1, 0, 1, 1, 1, 0] <=  -1
[-1, 1, -1, 0, 1, 1, 0, -1] <=  -2
[0, -1, 1, 0, -1, -1, 1, -1] <=  -3
[-3, 1